In [1]:
'''
3D Voxel Net
原始的版本是用于3D detection的（比如自动驾驶中的目标检测），特点是用到了点云体素化。
原始模型将空间分区后对点云采样来得到均匀的分区体素数据，涉及一个叫做VFE层链的关键结构；
而我们这里（因为数据量小）直接沿用了3D CNN的点云体素化部分，因此没有这个结构。
原始模型中间卷积层较多，我们的较少。原始模型的输出端是探测模型，而我们改成回归/分类模型。
总之，特点改没了...

模型结构：在原本3D CNN的最前端加入了一个3D卷积层和一个池化层。

实验表明，
在输出表现相似时，此模型的Loss相较其他模型的偏小。
在30轮训练后，此模型的表现也优于其他模型。
'''

'\n3D Voxel Net\n同表现下Loss偏小\n'

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
import trimesh
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [3]:
# 加载和归一化模型
def load_mesh(file_path):
    mesh = trimesh.load(file_path)
    # 如果加载的是Scene对象，转换为Trimesh对象
    if isinstance(mesh, trimesh.Scene):
        mesh = mesh.dump(concatenate=True)
    return mesh

def normalize_mesh(mesh):
    # 将网格中心平移到原点
    mesh.apply_translation(-mesh.centroid)
    # 计算缩放比例
    scale_factor = 1.0 / max(mesh.extents)
    # 缩放网格
    mesh.apply_scale(scale_factor)
    return mesh

def mesh_to_voxel(mesh, voxel_size=32):
    # 将网格体素化
    voxelized_mesh = mesh.voxelized(pitch=1.0 / voxel_size)
    # 确保体素矩阵的形状为 (voxel_size, voxel_size, voxel_size)
    voxel_matrix = voxelized_mesh.matrix
    padded_matrix = np.zeros((voxel_size, voxel_size, voxel_size), dtype=voxel_matrix.dtype)
    shape = np.minimum(voxel_matrix.shape, (voxel_size, voxel_size, voxel_size))
    padded_matrix[:shape[0], :shape[1], :shape[2]] = voxel_matrix[:shape[0], :shape[1], :shape[2]]
    return padded_matrix

# 读取评分
def read_scores(score_file_path):
    indices = []
    scores = []
    with open(score_file_path, 'r') as f:
        for line in f:
            index, score = line.strip().split()
            indices.append(int(index))
            scores.append(float(score))
    return indices, scores  

# 定义数据集
class TeethDataset(Dataset):
    def __init__(self, score_file_path, voxel_size=32):
        self.indices, self.scores = read_scores(score_file_path)
        self.voxel_size = voxel_size

    def __len__(self):
        return len(self.scores)

    def __getitem__(self, idx):
        index = self.indices[idx]
        score = self.scores[idx]
        mesh_path = f'{index}.obj'
        mesh = normalize_mesh(load_mesh(mesh_path))
        voxel_data = mesh_to_voxel(mesh, self.voxel_size)
        return torch.tensor(voxel_data[np.newaxis, :], dtype=torch.float32), torch.tensor([score], dtype=torch.float32)

# 创建数据集
score_file_path = 'scores.txt'
dataset = TeethDataset(score_file_path, voxel_size=32)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

In [4]:
# 定义VoxelNet模型
class VoxelNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VoxelNet, self).__init__()
        self.conv1 = nn.Conv3d(in_channels, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm3d(32)
        self.conv2 = nn.Conv3d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm3d(64)
        self.conv3 = nn.Conv3d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm3d(128)
        self.pool = nn.MaxPool3d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 4 * 4 * 4, 256)
        self.fc2 = nn.Linear(256, out_channels)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.pool(self.relu(self.bn3(self.conv3(x))))
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
# 创建模型
model = VoxelNet(in_channels=1, out_channels=1)
criterion = nn.MSELoss()  # 损失函数
optimizer = optim.Adam(model.parameters(), lr=0.001)  # 学习率和优化器

# 训练模型
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloader)}")

# 模型评估
model.eval()
test_loss = 0.0
all_targets = []
all_outputs = []
with torch.no_grad():
    for inputs, targets in dataloader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()
        all_targets.extend(targets.numpy().flatten())
        all_outputs.extend(outputs.numpy().flatten())
print(f"Test Loss: {test_loss / len(dataloader)}")


Epoch 1/30, Loss: 580.6649059998362
Epoch 2/30, Loss: 249.92723399714419
Epoch 3/30, Loss: 280.36314994410463
Epoch 4/30, Loss: 304.7574296248587
Epoch 5/30, Loss: 221.69632670753882
Epoch 6/30, Loss: 196.28072888600198
Epoch 7/30, Loss: 215.56352645472475
Epoch 8/30, Loss: 162.52317417295356
Epoch 9/30, Loss: 152.01632891203226
Epoch 10/30, Loss: 132.53206815217672
Epoch 11/30, Loss: 146.04363108308692
Epoch 12/30, Loss: 104.86361187382748
Epoch 13/30, Loss: 118.8970862940738
Epoch 14/30, Loss: 79.11647297206677
Epoch 15/30, Loss: 65.28073387396962
Epoch 16/30, Loss: 67.94036995737176
Epoch 17/30, Loss: 47.25428417481874
Epoch 18/30, Loss: 60.046128169486394
Epoch 19/30, Loss: 48.87576520442963
Epoch 20/30, Loss: 65.80777599937038
Epoch 21/30, Loss: 48.018507289259055
Epoch 22/30, Loss: 41.614423519686646
Epoch 23/30, Loss: 40.4859299659729
Epoch 24/30, Loss: 31.632226661631936
Epoch 25/30, Loss: 22.76944345549533
Epoch 26/30, Loss: 28.21058379976373
Epoch 27/30, Loss: 23.153757151804

In [6]:
# 计算评价指标
mse = mean_squared_error(all_targets, all_outputs)
rmse = np.sqrt(mse)
mae = mean_absolute_error(all_targets, all_outputs)
r2 = r2_score(all_targets, all_outputs)
mape = np.mean(np.abs((np.array(all_targets) - np.array(all_outputs)) / np.array(all_targets))) * 100

n = len(all_targets)
k = sum(p.numel() for p in model.parameters())
aic = n * np.log(mse) + 2 * k
bic = n * np.log(mse) + k * np.log(n)

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R^2: {r2}")
print(f"MAPE: {mape}%")
print(f"AIC: {aic}")
print(f"BIC: {bic}")

MSE: 18.85594367980957
RMSE: 4.342343330383301
MAE: 3.826805830001831
R^2: 0.926137306490977
MAPE: 5.942206084728241%
AIC: 4751802.524220467
BIC: 11904111.590334352


In [8]:
# 对150个预备体进行评分
model.eval()
scores = []
with torch.no_grad():
    for i in range(len(dataset)):
        inputs, _ = dataset[i]  # 忽略目标评分，只取输入数据
        inputs = inputs.unsqueeze(0)  # 添加批次维度
        output = model(inputs)
        scores.append(output.item())

# 输出150个预备体的评分
_, Real_Scores = read_scores(score_file_path)

for i, score in enumerate(scores):
    print(f'Tooth {i+1}: Score = {score}', f'Err = {score-Real_Scores[i]}' )


Tooth 1: Score = 82.79110717773438 Err = 7.791107177734375
Tooth 2: Score = 75.12904357910156 Err = 7.1290435791015625
Tooth 3: Score = 70.1851806640625 Err = 0.1851806640625
Tooth 4: Score = 70.67601013183594 Err = 1.6760101318359375
Tooth 5: Score = 79.29132843017578 Err = 3.2913284301757812
Tooth 6: Score = 69.51695251464844 Err = 5.5169525146484375
Tooth 7: Score = 72.08008575439453 Err = 3.0800857543945312
Tooth 8: Score = 73.22502136230469 Err = 0.2250213623046875
Tooth 9: Score = 85.05998229980469 Err = 8.059982299804688
Tooth 10: Score = 83.28384399414062 Err = 6.283843994140625
Tooth 11: Score = 55.87632369995117 Err = 1.8763236999511719
Tooth 12: Score = 65.6662368774414 Err = 3.6662368774414062
Tooth 13: Score = 61.8256950378418 Err = 2.825695037841797
Tooth 14: Score = 70.41460418701172 Err = 7.414604187011719
Tooth 15: Score = 61.0174560546875 Err = 4.0174560546875
Tooth 16: Score = 48.3412971496582 Err = 0.3412971496582031
Tooth 17: Score = 60.444576263427734 Err = 0.4445